# first iteration 

import * as vscode from 'vscode';

export function activate(context: vscode.ExtensionContext) {
    console.log('Congratulations, your extension is now active!');

    // Sample multiline text to insert
    const multilineText = `Line 1
Line 2
Line 3`;



    // Function to insert multiline text at the current cursor position
    const insertMultilineText = async (editor: vscode.TextEditor) => {
        const position = editor.selection.active;
        const newPosition = position.with(position.line, position.character);

        await editor.edit((editBuilder) => {
            editBuilder.insert(newPosition, multilineText);
        });

        const lines = multilineText.split('\n');
        const lastLine = lines[lines.length - 1];
        const endPosition = new vscode.Position(newPosition.line + lines.length - 1, lastLine.length);

        editor.selection = new vscode.Selection(endPosition, endPosition);
    };

    // Register a command to manually insert multiline text
    const disposableCommand = vscode.commands.registerCommand('trial.insertMultilineText', () => {
        const editor = vscode.window.activeTextEditor;
        if (editor) {
            insertMultilineText(editor);
        } else {
            vscode.window.showErrorMessage('No active text editor found.');
        }
    });


}

export function deactivate() {}


second iteration

import * as vscode from 'vscode';

export function activate(context: vscode.ExtensionContext) {
    console.log('Congratulations, your extension is now active!');

    const multilineText = `Line 1\nLine 2\nLine 3`;

    const insertMultilineText = async (editor: vscode.TextEditor) => {
        const position = editor.selection.active;
        const newPosition = position.with(position.line, position.character);
 
        await editor.edit((editBuilder) => {
            editBuilder.insert(newPosition, multilineText);
        });
 
        const lines = multilineText.split('\n');
        const lastLine = lines[lines.length - 1];
        const endPosition = new vscode.Position(newPosition.line + lines.length - 1, lastLine.length);
 
        editor.selection = new vscode.Selection(endPosition, endPosition);
    };
 
    const disposableCommand = vscode.commands.registerCommand('trial.insertMultilineText', () => {
        const editor = vscode.window.activeTextEditor;
        if (editor) {
            insertMultilineText(editor);
        } else {
            vscode.window.showErrorMessage('No active text editor found.');
        }
    });
 
    context.subscriptions.push(disposableCommand);
 
    const decorationType = vscode.window.createTextEditorDecorationType({
        after: {
            contentText: '⏳',
            color: 'rgba(255,0,0,0.8)'
        }
    });
 
    let timeout: NodeJS.Timeout | undefined;
 
    vscode.window.onDidChangeTextEditorSelection((event) => {
        if (timeout) {
            clearTimeout(timeout);
        }
 
        timeout = setTimeout(() => {
            const editor = event.textEditor;
            const position = editor.selection.active;
            const decoration = {
                range: new vscode.Range(position, position),
                renderOptions: {
                    after: {
                        contentText: multilineText,
                        margin: '0 0 0 1em',
                        color: 'gray',
                        backgroundColor: 'lightyellow'
                    }
                }
            };
 
            editor.setDecorations(decorationType, [decoration]);
        }, 1000);
    });
}

export function deactivate() {}


import * as vscode from 'vscode';

export function activate(context: vscode.ExtensionContext) {
    context.subscriptions.push(vscode.languages.registerInlineCompletionItemProvider({ scheme: 'file', language: '*' }, {
        provideInlineCompletionItems(document: vscode.TextDocument, position: vscode.Position, context: vscode.InlineCompletionContext, token: vscode.CancellationToken) {
            const linePrefix = document.lineAt(position).text.substr(0, position.character);
            if (linePrefix.endsWith(' ')) {
                return [new vscode.InlineCompletionItem('This is a multiline\nsuggestion that appears\nwhen space is pressed.')];
            }
            return [];
        }
    }));

    
}

export function deactivate() {}


In [ ]:
const vscode = require('vscode');

let lastKeyPressTime = 0;

/**
 * @param {vscode.ExtensionContext} context
 */
function activate(context) {
  let disposable = vscode.commands.registerCommand('trial.insertMultilineText', function () {
    const editor = vscode.window.activeTextEditor;
    if (editor) {
      const currentTime = new Date().getTime();
      if (currentTime - lastKeyPressTime < 500) {
        // Less than half a second since last key press, insert tab
        editor.insertSnippet(new vscode.SnippetString('\t'));
      } else {
        // More than half a second, insert multiline text
        const position = editor.selection.active;
        const snippet = new vscode.SnippetString("Suggestion Line 1\nSuggestion Line 2\nSuggestion Line 3");
        editor.insertSnippet(snippet, position);
      }
      lastKeyPressTime = currentTime;
    }
  });

  context.subscriptions.push(disposable);
}

exports.activate = activate;

function deactivate() {}

module.exports = {
  activate,
  deactivate
};


# Funtioning snippet adder

import Groq from 'groq-sdk';


const groq = new Groq();

let generatedSuggestion = '';

export const getGeneratedSuggestion = () => generatedSuggestion;

export const setGeneratedSuggestion = (suggestion: string) => {
    generatedSuggestion = suggestion;
};

export const getCompletion = async (text: string) => {
    try {
        const response = await groq.chat.completions.create({
            messages: [
                { role: 'system', content: 'You are given an incomplete code for which you only return code completion.' },
                { role: 'user', content: text }
            ],
            model: 'llama3-8b-8192',
            temperature: 0.5,
            max_tokens: 1024,
            top_p: 1,
            stop: null,
            stream: false
        });
        return response.choices[0]?.message?.content || '';
    } catch (error) {
        console.error('Error getting Groq completion:', error);
        return '';
    }
};


import { getGeneratedSuggestion, setGeneratedSuggestion, getCompletion } from './suggestionUpdater';
import vscode from 'vscode';
let lastKeyPressTime = 0;

/**
 * @param {vscode.ExtensionContext} context
 */
function activate(context: vscode.ExtensionContext) {
    try {
        context.subscriptions.push(vscode.languages.registerInlineCompletionItemProvider({ scheme: 'file', language: '*' }, {
            provideInlineCompletionItems(document: vscode.TextDocument, position: vscode.Position, context: vscode.InlineCompletionContext, token: vscode.CancellationToken) {
                try {
                    const linePrefix = document.lineAt(position).text.substr(0, position.character);
                    if (linePrefix.endsWith(' ')) {
                        return {
                            items: [
                                new vscode.InlineCompletionItem(getGeneratedSuggestion())
                            ]
                        };
                    }
                    return { items: [] };
                } catch (error: any) {
                    vscode.window.showErrorMessage(`Error providing inline completion items: ${error.message}`);
                    return { items: [] };
                }
            }
        }));

        let disposable = vscode.commands.registerCommand('trial.insertMultilineText', function () {
            try {
                const editor = vscode.window.activeTextEditor;
                if (editor) {
                    const currentTime = new Date().getTime();
                    const timeSinceLastKeyPress = currentTime - lastKeyPressTime;
                    vscode.window.showInformationMessage(`Time since last key press: ${timeSinceLastKeyPress}ms`);
                    if (timeSinceLastKeyPress < 2000) {
                        // Less than two seconds since last key press, insert tab
                        vscode.window.showInformationMessage('Inserting tab');
                        editor.insertSnippet(new vscode.SnippetString('\t'));
                    } else {
                        // More than two seconds, insert multiline text
                        vscode.window.showInformationMessage('Inserting multiline text');
                        const position = editor.selection.active;
                        const snippet = new vscode.SnippetString(getGeneratedSuggestion());
                        editor.insertSnippet(snippet, position);
                    }
                    lastKeyPressTime = currentTime;
                }
            } catch (error: any) {
                vscode.window.showErrorMessage(`Error inserting multiline text: ${error.message}`);
            }
        });
        context.subscriptions.push(disposable);

        // Add an event listener for text document changes
        vscode.workspace.onDidChangeTextDocument(async event => {
            const editor = vscode.window.activeTextEditor;
            if (editor && event.document === editor.document) {
                const position = editor.selection.active;
                const textBeforeCursor = editor.document.getText(new vscode.Range(new vscode.Position(0, 0), position));
                const groqCompletion = await getCompletion(textBeforeCursor);
                setGeneratedSuggestion(groqCompletion);
            }
        });
    } catch (error: any) {
        vscode.window.showErrorMessage(`Error activating extension: ${error.message}`);
    }
}

exports.activate = activate;

function deactivate() {}

module.exports = {
    activate,
    deactivate
};
